In [ ]:
import torch
from Agent2 import Agent
from Product import Product
from Belt import Belt
from Buffer import Buffer
from Pallet import Pallet

import random
from matplotlib import pyplot as plt
import numpy as np
import math

import threading
import time
import csv


class Time():
   def __init__(self,time:int) -> None:
      self.time = time
   
   def increaseTime(self):
      self.time += 1

globalTime = Time(0)

numberOfEpisodes = 2500

belt = Belt(1)
buffer = Buffer(2,3)
pallet = Pallet(3)
agent = Agent(belt=belt,buffer=buffer,pallet=pallet, globalTime = globalTime)


deltaTStart = 0
deltaTEnd = 2

weightStart = 1
weightEnd = 8

exitFlag = 0

episode = 1

with open("log.txt", "w") as log:
   log.write(" ------------------ = New Trial =  -------------------\n")

class Conveyor (threading.Thread):
   def __init__(self, threadID, name):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
   def run(self):
      print("Conveyor belt started ... " , self.name)
      feedProduct(self.name, 0.05)
      print("Conveyor belt stopped ... " , self.name)

def feedProduct(threadName, delay):
   while True:
      if exitFlag:
         return
      time.sleep(delay)
      if  not belt.isFull():
         belt.addProduct(Product(arrivalTime = globalTime.time+random.randint(deltaTStart,deltaTEnd), weight = random.randint(weightStart,weightEnd)))

factory = Conveyor(1, "Belt-Thread")

factory.start()

with open("log.csv", "a") as log:
   log.write("Episode,Episode Reward,Episode Steps,Normalized Episode Reward,Epsilon,Final Pallet Reward,Quartile Reward Per Step,Shipped Pallet\n")

while episode <= numberOfEpisodes:
   
   time.sleep(0.05)
   
   if episode >= numberOfEpisodes:
      break
   
   ### agent do action here (if needed!)
   agent.learn(episode)
   
   torch.save(agent.act_net.state_dict(), "./act_net_done.pth")
   
   normalReward = np.nanquantile(agent.actionRewards,0.15) if not math.isnan(np.nanquantile(agent.actionRewards,0.15)) else agent.defaultActionReward
   
   if episode % 200 == 0:
      print("-------------")
      print("Episode: ",episode)
      print("Total reward: ", agent.episodeRewards[-1]/agent.episodeSteps[-1])
      print("Quartile Reward: ",normalReward)
      print("Final Pallet Reward: ", agent.lastRWeight)
      print("----------------------")
   
   with open("log.csv", "a") as log:
      writer = csv.writer(log, delimiter=',' , lineterminator='\n')
      writer.writerow([episode, agent.episodeRewards[-1], agent.episodeSteps[-1], agent.episodeRewards[-1]/agent.episodeSteps[-1], agent.epsilon,agent.lastRWeight,normalReward,agent.lastPallet])
   
   episode += 1
   # belt.empty()
   # buffer.empty()
   # pallet.empty()
   
   ## tille here

   ### update graphic canvas here
   ## till here 

exitFlag = True


average_rewards  = np.array(agent.episodeRewards)/np.array(agent.episodeSteps)
plt.plot(average_rewards, "-o", label="Average Reward")
plt.ylabel("Average Reward")
plt.xlabel("Episodes")
plt.legend('Average Reward')
plt.savefig("rewards.png", dpi=300)
# print("Average Reward: ", len(average_rewards))




In [22]:
from matplotlib.pyplot import get_current_fig_manager, step
import random
import math

from numpy.core.fromnumeric import product
from numpy.lib.function_base import copy

from Product import Product
from Belt import Belt
from Buffer import Buffer
from Pallet import Pallet
from Net2 import Net
from copy import copy as clone

from collections import defaultdict

import argparse
import pickle
from collections import namedtuple
from itertools import count

import os, time
import numpy as np
import matplotlib.pyplot as plt
import math

import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal, Categorical
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

In [102]:
if agent.learn_counter % agent.Q_network_evaluation == 0:
    agent.target_net.load_state_dict(agent.act_net.state_dict())
agent.learn_counter += 1

sample_index = np.random.choice(agent.capacity, agent.batch_size)
batch_memory = agent.memory[sample_index, :]
batch_state = torch.FloatTensor(batch_memory[:, :agent.num_state_features])
#note that the action must be a int
batch_action = torch.LongTensor(batch_memory[:, agent.num_state_features:agent.num_state_features+1].astype(int))
batch_reward = torch.FloatTensor(batch_memory[:, agent.num_state_features+1: agent.num_state_features+2])
batch_next_state = torch.FloatTensor(batch_memory[:, -agent.num_state_features:])

q_eval = agent.act_net(batch_state).gather(1, batch_action)
q_next = agent.target_net(batch_next_state).detach()
q_target = batch_reward + agent.gamma*q_next.max(1)[0].view(agent.batch_size, 1) - agent.target_net(batch_state).gather(1, batch_action)

# loss = agent.loss(q_eval, q_target)
# agent.optimizer.zero_grad()
# loss.backward()
# agent.optimizer.step()

In [13]:
sample_index = np.random.choice(agent.capacity, agent.batch_size)
batch_memory = agent.memory[sample_index, :]
print(len(sample_index))

512


In [19]:
print(agent.num_state_features)
len(agent.memory[0,:])
print(batch_memory[511])

20
[  0.    3.    0.    4.    0.    3.    0.    0.    0.    0.    0.    0.
   0.    8.    0.    8.    0.    0.    0.    0.   13.  -41.6   0.    3.
   0.    4.    0.    3.    0.    0.    0.    0.    0.    0.    0.    8.
   0.    8.    0.    0.    0.    0. ]


In [69]:
torch.FloatTensor(agent.act_net(batch_state)[0])#.gather(0,batch_action[0])

tensor([1881193.1250, 1816905.8750, 1657790.7500, 1802376.5000, 1751255.1250,
        1804798.8750, 1711546.7500, 1636652.2500, 1705764.7500, 1822496.6250,
        1758784.5000, 1788910.1250, 1720412.5000, 1795192.5000],
       grad_fn=<AliasBackward0>)

In [ ]:
q_next[:].max(1)[0].view(agent.batch_size, 1)

In [104]:
# q_eval[0]
q_target[0]

tensor([1773839.8750])